# Dog vs AI‑Generated Dog Image Classifier
본 노트북은 `dog_classifier.py` 스크립트를 Jupyter Notebook 형식으로 변환한 것입니다. AI로 생성된 강아지 이미지와 실제 강아지 이미지를 분류하는 ResNet‑18 기반 모델의 데이터 전처리, 학습, 평가 과정을 단계별로 실행할 수 있습니다.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report

def print_gpu_info():
    """GPU 정보 출력"""
    if torch.cuda.is_available():
        print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        print(f"Free GPU Memory: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    else:
        print("CUDA is not available. Using CPU.")

In [ ]:
# 데이터 경로 설정
DATA_ROOT = os.path.join(os.path.expanduser("~"), "workspace", "projects", "intotheai", "data", "Dogs Vs AiDogs", "Dogs Vs AiDogs")
TRAIN_DIR = os.path.join(DATA_ROOT, "train", "images")
VALID_DIR = os.path.join(DATA_ROOT, "valid", "images")
TEST_DIR = os.path.join(DATA_ROOT, "test", "images")

In [ ]:
# 이미지 변환 정의
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# 커스텀 데이터셋 클래스 정의
class DogDataset(Dataset):
    def __init__(self, dir_path, transform=None):
        self.dir_path = dir_path
        self.transform = transform
        self.images = [f for f in os.listdir(dir_path) if f.endswith('.jpg')]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.dir_path, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = 0 if img_name.startswith('ai_') else 1
        return image, label

In [ ]:
# 데이터셋 및 데이터로더 생성 함수
def get_dataloaders(batch_size=32):
    train_dataset = DogDataset(TRAIN_DIR, transform=train_transform)
    valid_dataset = DogDataset(VALID_DIR, transform=test_transform)
    test_dataset = DogDataset(TEST_DIR, transform=test_transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)
    return train_loader, valid_loader, test_loader

In [ ]:
# 모델 정의 (ResNet‑18 기반)
def get_model():
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    return model

In [ ]:
# 모델 학습 함수
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    model.to(device)
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    best_val_acc = 0.0
    model_save_path = 'best_dog_classifier.pth'
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]'):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_losses.append(running_loss / len(train_loader.dataset))
        train_accs.append(100 * correct / total)
        model.eval(); running_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(valid_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Valid]'):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_losses.append(running_loss / len(valid_loader.dataset))
        val_accs.append(100 * correct / total)
        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accs[-1]:.2f}%, Valid Loss: {val_losses[-1]:.4f}, Valid Acc: {val_accs[-1]:.2f}%')
        if val_accs[-1] > best_val_acc:
            best_val_acc = val_accs[-1]
            torch.save(model.state_dict(), model_save_path)
            print(f'Model saved with validation accuracy: {best_val_acc:.2f}%')
    return model, {'train_loss': train_losses, 'val_loss': val_losses, 'train_acc': train_accs, 'val_acc': val_accs}

In [ ]:
# 평가 및 시각화 함수들
def evaluate_model(model, test_loader):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device); model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    accuracy = sum(1 for i, j in zip(y_true, y_pred) if i == j) / len(y_true)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
    cm = confusion_matrix(y_true, y_pred)
    print('Confusion Matrix:')
    print(cm)
    print('Classification Report:')
    print(classification_report(y_true, y_pred, target_names=['AI Generated', 'Real']))
    return accuracy, cm

def plot_training_history(history):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss over epochs'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy over epochs'); plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend()
    plt.tight_layout(); plt.show()

def predict_and_visualize(model, test_loader, num_samples=10):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device); model.eval()
    inv_normalize = transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225], std=[1/0.229, 1/0.224, 1/0.225])
    images, labels, preds = [], [], []
    with torch.no_grad():
        for batch_imgs, batch_labels in test_loader:
            batch_imgs, batch_labels = batch_imgs.to(device), batch_labels.to(device)
            outputs = model(batch_imgs)
            _, batch_preds = torch.max(outputs, 1)
            for i in range(batch_imgs.size(0)):
                images.append(inv_normalize(batch_imgs[i]).cpu())
                labels.append(batch_labels[i].item())
                preds.append(batch_preds[i].item())
                if len(images) >= num_samples:
                    break
            if len(images) >= num_samples:
                break
    class_names = ['AI Generated', 'Real']
    fig, axes = plt.subplots(2, 5, figsize=(15,6)); axes = axes.flatten()
    for i, (img, label, pred) in enumerate(zip(images, labels, preds)):
        img = img.permute(1,2,0).numpy(); img = np.clip(img, 0, 1)
        axes[i].imshow(img)
        color = 'green' if label == pred else 'red'
        axes[i].set_title(f'True: {class_names[label]}\nPred: {class_names[pred]}', color=color)
        axes[i].axis('off')
    plt.tight_layout(); plt.show()

In [ ]:
# 클래스별 이미지 개수 계산 함수
def calculate_class_distribution(dataset):
    labels = [0 if img.startswith('ai_') else 1 for img in dataset.images]
    return labels.count(0), labels.count(1)

In [ ]:
# ===================== 실행 예시 =====================
if __name__ == '__main__':
    BATCH_SIZE = 512
    NUM_EPOCHS = 10
    print_gpu_info()
    train_loader, valid_loader, test_loader = get_dataloaders(batch_size=BATCH_SIZE)
    print('Dataset Statistics:')
    print(f"Training - AI Generated: {calculate_class_distribution(train_loader.dataset)[0]}, Real: {calculate_class_distribution(train_loader.dataset)[1]}")
    model = get_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    trained_model, history = train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=NUM_EPOCHS)
    plot_training_history(history)
    model.load_state_dict(torch.load('best_dog_classifier.pth'))
    evaluate_model(model, test_loader)
    predict_and_visualize(model, test_loader, num_samples=10)
    print('Training and evaluation completed!')
    torch.backends.cudnn.benchmark = True  # GPU 성능 최적화